In [108]:
# inporting libraries
import numpy as np
import pandas as pd
import tensorflow as tf

In [109]:
tf.__version__

'2.9.1'

In [110]:
## Part 1 - Data Preprocessing
## importing dataset
dataset = pd.read_csv('Churn_Modelling.csv')
x = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [111]:
print(x)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [112]:
print(y)

[1 0 1 ... 1 1 0]


In [113]:
### Encoding categorical data
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
x[:,2]=le.fit_transform(x[:,2])

In [114]:

print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [115]:
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers= [('encoder',OneHotEncoder(),[1])],remainder='passthrough')
x =np.array(ct.fit_transform(x))

In [116]:
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [117]:
### Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)


In [118]:
### Feature Scaling
# future scaling we used standardization best for everyone(produce result b/t -3 - 3) 
# (other is normalization best of normal eq) produce result b/t 0 - 1 
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
x_train=ss.fit_transform(x_train)
x_test=ss.transform(x_test)

In [119]:
## Part 2 - Building the ANN
# ### Initializing the ANN
ann =tf.keras.models.Sequential()

In [120]:
### Adding the input layer and the first hidden layer
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

In [121]:
### Adding the second hidden layer
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

In [122]:
### Adding the output layer
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

In [123]:
## Part 3 - Training the ANN
### Compiling the ANN
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy']) #using stochastic gradient descent  
# for binary dependent variable we use binary_crossentropy
# for categorical dependent variable we use  categorical_crossentropy

In [124]:
### Training the ANN on the Training set
ann.fit(x_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
250/250 [==============================] - 2s 3ms/step - loss: 0.5936 - accuracy: 0.7751
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4904 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4463 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4229 - accuracy: 0.8076
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4110 - accuracy: 0.8199
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4019 - accuracy: 0.8276
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3937 - accuracy: 0.8320
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3853 - accuracy: 0.8331
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3773 - accuracy: 0.8403
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3701 - accura

## Part 4 - Making the predictions and evaluating the model
### Predicting the result of a single observation


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

In [125]:
print(ann.predict(ss.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]]))>0.5)

1/1 [==============================] - 0s 130ms/step
[[False]]


In [126]:
### Predicting the Test set results
y_pred=ann.predict(x_test)
y_pred=(y_pred>0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 2ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [127]:
### Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm=confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[1502   93]
 [ 194  211]]


0.8565